In [1]:
import numpy as np
import pandas as pd 

x_train_reduced = pd.read_csv('x_train_reduced.csv')
x_test_reduced = pd.read_csv('x_test_reduced.csv')
y_train2 = pd.read_csv('y_train2.csv')

/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
x_train_reduced = x_train_reduced.loc[:, ~x_train_reduced.columns.str.contains('^Unnamed')]
x_test_reduced = x_test_reduced.loc[:, ~x_test_reduced.columns.str.contains('^Unnamed')]

In [12]:
y_train3 = y_train2.drop(labels=['fullVisitorId'], axis=1)

In [19]:
# Hyperparameter Tuning
# For higher accuracy, tuning max_bin, learning_rate, num_leaves, max_depth, min_data_in_leaf
from sklearn.model_selection import GridSearchCV

In [37]:
%%time
# GBDT
estimator = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', bagging_fraction = 0.7,
        feature_fraction = 0.7,
        bagging_frequency = 6,
        bagging_seed = 42,
        seed = 42)

param_grid = {
    'learning_rate': [0.0015, 0.0025, 0.005, 0.0075, 0.01],
    'num_leaves': [30, 40, 70, 100],
    'max_depth': [7, 10, 15, 20],
    'max_bin':[255, 300, 350],
    'min_data_in_leaf':[20, 50, 100]
    }

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(x_train_reduced, y_train3)

print(gbm.best_params_)

{'learning_rate': 0.01, 'max_bin': 300, 'max_depth': 15, 'min_data_in_leaf': 50, 'num_leaves': 100}
CPU times: user 7d 9h 58min 35s, sys: 3h 49min 13s, total: 7d 13h 47min 49s
Wall time: 15h 37min 13s


In [58]:
%%time
# Dart
estimator1 = lgb.LGBMRegressor(boosting_type='dart', objective='regression', 
                               max_bin=300, min_data_in_leaf=50, max_depth=15,
                               bagging_fraction = 0.7, feature_fraction = 0.7, bagging_frequency = 6, 
                               bagging_seed = 42, seed = 42)

param_grid_dart = {
    'learning_rate': [0.0025, 0.005, 0.01],
    'num_leaves': [40, 70, 100],
    'max_drop':[30, 50, 70],
    'drop_rate':[0.05, 0.1, 0.2],
    'skip_drop':[0.3, 0.5, 0.7]
    }

gbm_dart = GridSearchCV(estimator1, param_grid_dart, cv=3)
gbm_dart.fit(x_train_reduced, y_train3)

print(gbm_dart.best_params_)

{'drop_rate': 0.05, 'learning_rate': 0.01, 'max_drop': 30, 'num_leaves': 100, 'skip_drop': 0.7}
CPU times: user 3d 21min 35s, sys: 1h 17min 33s, total: 3d 1h 39min 8s
Wall time: 5h 53min 43s


In [ ]:
# Dart performance is no better than gbdt

In [65]:
%%time
# Further increase learning_rate and num_leaves, since other parameters fall in middle of the value interval 
# GBDT
estimator2 = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', bagging_fraction = 0.7,
        feature_fraction = 0.7,
        bagging_frequency = 6,
        bagging_seed = 42,
        seed = 42,
        max_bin=300,
        max_depth=15,
        min_data_in_leaf=50)

param_grid2 = {
    'learning_rate': [0.01, 0.015, 0.02, 0.03],
    'num_leaves': [100, 150, 200, 250]
    }

gbm2 = GridSearchCV(estimator2, param_grid2, cv=3)
gbm2.fit(x_train_reduced, y_train3)

print(gbm2.best_params_)

{'learning_rate': 0.03, 'num_leaves': 250}
CPU times: user 19h 57min 36s, sys: 13min 29s, total: 20h 11min 5s
Wall time: 1h 12min 13s


In [70]:
%%time
# Continue increase learning_rate and num_leaves 
# GBDT
estimator3 = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', bagging_fraction = 0.7,
        feature_fraction = 0.7,
        bagging_frequency = 6,
        bagging_seed = 42,
        seed = 42,
        max_bin=300,
        max_depth=15)

param_grid3 = {
    'learning_rate': [0.03, 0.05, 0.1, 0.15],
    'num_leaves': [250, 350, 500, 750],
    'min_data_in_leaf': [20, 50, 100, 200]
    }

gbm3 = GridSearchCV(estimator3, param_grid3, cv=3)
gbm3.fit(x_train_reduced, y_train3)

print(gbm3.best_params_)

{'learning_rate': 0.1, 'min_data_in_leaf': 50, 'num_leaves': 500}
CPU times: user 3d 3h 6min 53s, sys: 1h 2min 5s, total: 3d 4h 8min 59s
Wall time: 4h 25min 47s


In [97]:
%%time
# Final fine-tuning
estimator4 = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', bagging_fraction = 0.7,
        feature_fraction = 0.7,
        bagging_frequency = 6,
        bagging_seed = 42,
        seed = 42,
        max_bin=300,
        max_depth=15,
        min_data_in_leaf=50)

param_grid4 = {
    'learning_rate': [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12],
    'num_leaves': [250, 280, 310, 340, 370, 400, 430, 460, 500, 530]
    }

gbm4 = GridSearchCV(estimator4, param_grid4, cv=3)
gbm4.fit(x_train_reduced, y_train3)

print(gbm4.best_params_)

{'learning_rate': 0.1, 'num_leaves': 500}
CPU times: user 2d 10h 54min 54s, sys: 1h 27min 54s, total: 2d 12h 22min 49s
Wall time: 3h 49min 57s


In [100]:
%%time
num_round = 7000
params = {"objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 250,
        "max_depth" : 15,
        "boosting" : "gbdt",
        "min_data_in_leaf": 50,
        "max_bin": 300,
        "learning_rate" : 0.03,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.7,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "seed": 42}

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 15 µs


In [101]:
#Output

x_train, x_val, y_train, y_val = train_test_split(x_train_reduced, y_train3, test_size=0.1, random_state=1)
lgb_train_data = lgb.Dataset(x_train, label=y_train)
lgb_val_data = lgb.Dataset(x_val, label=y_val)
rmse_val = lgb.train(params, lgb_train_data, num_round, early_stopping_rounds=20, 
                         valid_sets=[lgb_train_data, lgb_val_data])
y_pred_train = rmse_val.predict(x_train, num_iteration=rmse_val.best_iteration)
y_pred_val = rmse_val.predict(x_val, num_iteration=rmse_val.best_iteration)
rmse_v = np.sqrt(mean_squared_error(y_val, y_pred_val))
rmse_t = np.sqrt(mean_squared_error(y_train, y_pred_train))

[1]	training's rmse: 1.83295	valid_1's rmse: 1.83185
Training until validation scores don't improve for 20 rounds.
[2]	training's rmse: 1.79279	valid_1's rmse: 1.79351
[3]	training's rmse: 1.75559	valid_1's rmse: 1.75765
[4]	training's rmse: 1.71794	valid_1's rmse: 1.72139
[5]	training's rmse: 1.68117	valid_1's rmse: 1.68643
[6]	training's rmse: 1.64585	valid_1's rmse: 1.65301
[7]	training's rmse: 1.61238	valid_1's rmse: 1.62118
[8]	training's rmse: 1.57942	valid_1's rmse: 1.59034
[9]	training's rmse: 1.54748	valid_1's rmse: 1.5605
[10]	training's rmse: 1.51606	valid_1's rmse: 1.53106
[11]	training's rmse: 1.48551	valid_1's rmse: 1.5025
[12]	training's rmse: 1.45699	valid_1's rmse: 1.47535
[13]	training's rmse: 1.4284	valid_1's rmse: 1.44877
[14]	training's rmse: 1.40142	valid_1's rmse: 1.42395
[15]	training's rmse: 1.37501	valid_1's rmse: 1.39978
[16]	training's rmse: 1.34907	valid_1's rmse: 1.37528
[17]	training's rmse: 1.32408	valid_1's rmse: 1.3523
[18]	training's rmse: 1.30036	val

[148]	training's rmse: 0.510894	valid_1's rmse: 0.728507
[149]	training's rmse: 0.509308	valid_1's rmse: 0.727561
[150]	training's rmse: 0.50772	valid_1's rmse: 0.726941
[151]	training's rmse: 0.506273	valid_1's rmse: 0.726558
[152]	training's rmse: 0.504789	valid_1's rmse: 0.725996
[153]	training's rmse: 0.503138	valid_1's rmse: 0.725102
[154]	training's rmse: 0.501728	valid_1's rmse: 0.724381
[155]	training's rmse: 0.500283	valid_1's rmse: 0.72359
[156]	training's rmse: 0.498838	valid_1's rmse: 0.722892
[157]	training's rmse: 0.497483	valid_1's rmse: 0.722259
[158]	training's rmse: 0.496015	valid_1's rmse: 0.721615
[159]	training's rmse: 0.494548	valid_1's rmse: 0.720934
[160]	training's rmse: 0.493198	valid_1's rmse: 0.720367
[161]	training's rmse: 0.49172	valid_1's rmse: 0.719707
[162]	training's rmse: 0.490165	valid_1's rmse: 0.718956
[163]	training's rmse: 0.488672	valid_1's rmse: 0.718337
[164]	training's rmse: 0.487412	valid_1's rmse: 0.717727
[165]	training's rmse: 0.485966	va

[294]	training's rmse: 0.367968	valid_1's rmse: 0.672993
[295]	training's rmse: 0.367286	valid_1's rmse: 0.672731
[296]	training's rmse: 0.366708	valid_1's rmse: 0.672581
[297]	training's rmse: 0.366039	valid_1's rmse: 0.672359
[298]	training's rmse: 0.365341	valid_1's rmse: 0.672151
[299]	training's rmse: 0.364796	valid_1's rmse: 0.671978
[300]	training's rmse: 0.364223	valid_1's rmse: 0.671812
[301]	training's rmse: 0.363599	valid_1's rmse: 0.671639
[302]	training's rmse: 0.362834	valid_1's rmse: 0.671434
[303]	training's rmse: 0.362077	valid_1's rmse: 0.67118
[304]	training's rmse: 0.361492	valid_1's rmse: 0.671089
[305]	training's rmse: 0.360928	valid_1's rmse: 0.670851
[306]	training's rmse: 0.360388	valid_1's rmse: 0.670713
[307]	training's rmse: 0.359843	valid_1's rmse: 0.670573
[308]	training's rmse: 0.359209	valid_1's rmse: 0.670314
[309]	training's rmse: 0.358527	valid_1's rmse: 0.670108
[310]	training's rmse: 0.357949	valid_1's rmse: 0.669883
[311]	training's rmse: 0.357328	

[440]	training's rmse: 0.296483	valid_1's rmse: 0.652996
[441]	training's rmse: 0.296201	valid_1's rmse: 0.652965
[442]	training's rmse: 0.295797	valid_1's rmse: 0.652869
[443]	training's rmse: 0.295312	valid_1's rmse: 0.652687
[444]	training's rmse: 0.294999	valid_1's rmse: 0.652637
[445]	training's rmse: 0.294612	valid_1's rmse: 0.652537
[446]	training's rmse: 0.294233	valid_1's rmse: 0.652493
[447]	training's rmse: 0.293808	valid_1's rmse: 0.652353
[448]	training's rmse: 0.293298	valid_1's rmse: 0.652196
[449]	training's rmse: 0.29297	valid_1's rmse: 0.652131
[450]	training's rmse: 0.292638	valid_1's rmse: 0.652106
[451]	training's rmse: 0.292286	valid_1's rmse: 0.652026
[452]	training's rmse: 0.291898	valid_1's rmse: 0.651882
[453]	training's rmse: 0.29147	valid_1's rmse: 0.651707
[454]	training's rmse: 0.291164	valid_1's rmse: 0.65166
[455]	training's rmse: 0.290837	valid_1's rmse: 0.651565
[456]	training's rmse: 0.290393	valid_1's rmse: 0.651393
[457]	training's rmse: 0.290064	va

[585]	training's rmse: 0.248115	valid_1's rmse: 0.641192
[586]	training's rmse: 0.247872	valid_1's rmse: 0.641155
[587]	training's rmse: 0.24752	valid_1's rmse: 0.641118
[588]	training's rmse: 0.247281	valid_1's rmse: 0.641092
[589]	training's rmse: 0.247056	valid_1's rmse: 0.641082
[590]	training's rmse: 0.246844	valid_1's rmse: 0.641046
[591]	training's rmse: 0.24661	valid_1's rmse: 0.640984
[592]	training's rmse: 0.246347	valid_1's rmse: 0.640933
[593]	training's rmse: 0.24605	valid_1's rmse: 0.640847
[594]	training's rmse: 0.24581	valid_1's rmse: 0.640798
[595]	training's rmse: 0.245572	valid_1's rmse: 0.640769
[596]	training's rmse: 0.245335	valid_1's rmse: 0.640702
[597]	training's rmse: 0.245034	valid_1's rmse: 0.640633
[598]	training's rmse: 0.244723	valid_1's rmse: 0.640553
[599]	training's rmse: 0.244349	valid_1's rmse: 0.640476
[600]	training's rmse: 0.244098	valid_1's rmse: 0.640447
[601]	training's rmse: 0.243822	valid_1's rmse: 0.64035
[602]	training's rmse: 0.243595	vali

...

[4397]	training's rmse: 0.0345614	valid_1's rmse: 0.611091
[4398]	training's rmse: 0.0345521	valid_1's rmse: 0.611089
[4399]	training's rmse: 0.0345483	valid_1's rmse: 0.61109
[4400]	training's rmse: 0.0345419	valid_1's rmse: 0.611089
[4401]	training's rmse: 0.0345358	valid_1's rmse: 0.611089
[4402]	training's rmse: 0.0345299	valid_1's rmse: 0.61109
[4403]	training's rmse: 0.034514	valid_1's rmse: 0.611086
[4404]	training's rmse: 0.0345046	valid_1's rmse: 0.611085
[4405]	training's rmse: 0.0344934	valid_1's rmse: 0.611086
[4406]	training's rmse: 0.0344883	valid_1's rmse: 0.611084
[4407]	training's rmse: 0.034482	valid_1's rmse: 0.611085
[4408]	training's rmse: 0.0344758	valid_1's rmse: 0.611086
[4409]	training's rmse: 0.0344633	valid_1's rmse: 0.611085
[4410]	training's rmse: 0.0344545	valid_1's rmse: 0.611085
[4411]	training's rmse: 0.0344498	valid_1's rmse: 0.611084
[4412]	training's rmse: 0.034437	valid_1's rmse: 0.611084
[4413]	training's rmse: 0.0344294	valid_1's rmse: 0.611084
[4

[4537]	training's rmse: 0.0333361	valid_1's rmse: 0.611016
[4538]	training's rmse: 0.033324	valid_1's rmse: 0.611014
[4539]	training's rmse: 0.0333165	valid_1's rmse: 0.611013
[4540]	training's rmse: 0.0332967	valid_1's rmse: 0.611011
[4541]	training's rmse: 0.0332835	valid_1's rmse: 0.61101
[4542]	training's rmse: 0.0332703	valid_1's rmse: 0.611008
[4543]	training's rmse: 0.0332617	valid_1's rmse: 0.611008
[4544]	training's rmse: 0.0332531	valid_1's rmse: 0.611006
[4545]	training's rmse: 0.0332485	valid_1's rmse: 0.611006
[4546]	training's rmse: 0.0332417	valid_1's rmse: 0.611005
[4547]	training's rmse: 0.0332289	valid_1's rmse: 0.611005
[4548]	training's rmse: 0.0332215	valid_1's rmse: 0.611003
[4549]	training's rmse: 0.0332137	valid_1's rmse: 0.611003
[4550]	training's rmse: 0.0332033	valid_1's rmse: 0.611002
[4551]	training's rmse: 0.0331991	valid_1's rmse: 0.611001
[4552]	training's rmse: 0.0331842	valid_1's rmse: 0.610999
[4553]	training's rmse: 0.0331763	valid_1's rmse: 0.611
[4

[4677]	training's rmse: 0.032018	valid_1's rmse: 0.610907
[4678]	training's rmse: 0.0320126	valid_1's rmse: 0.610905
[4679]	training's rmse: 0.0319979	valid_1's rmse: 0.610907
[4680]	training's rmse: 0.0319912	valid_1's rmse: 0.610906
[4681]	training's rmse: 0.0319834	valid_1's rmse: 0.610905
[4682]	training's rmse: 0.0319751	valid_1's rmse: 0.610904
[4683]	training's rmse: 0.0319683	valid_1's rmse: 0.610904
[4684]	training's rmse: 0.031963	valid_1's rmse: 0.610904
[4685]	training's rmse: 0.031958	valid_1's rmse: 0.610903
[4686]	training's rmse: 0.0319487	valid_1's rmse: 0.6109
[4687]	training's rmse: 0.0319431	valid_1's rmse: 0.610899
[4688]	training's rmse: 0.0319228	valid_1's rmse: 0.610898
[4689]	training's rmse: 0.0319179	valid_1's rmse: 0.610897
[4690]	training's rmse: 0.0319097	valid_1's rmse: 0.610897
[4691]	training's rmse: 0.031901	valid_1's rmse: 0.610895
[4692]	training's rmse: 0.0318973	valid_1's rmse: 0.610895
[4693]	training's rmse: 0.0318931	valid_1's rmse: 0.610894
[46

[4818]	training's rmse: 0.0307935	valid_1's rmse: 0.610811
[4819]	training's rmse: 0.030791	valid_1's rmse: 0.610811
[4820]	training's rmse: 0.0307804	valid_1's rmse: 0.61081
[4821]	training's rmse: 0.0307686	valid_1's rmse: 0.610807
[4822]	training's rmse: 0.0307571	valid_1's rmse: 0.610808
[4823]	training's rmse: 0.0307499	valid_1's rmse: 0.610807
[4824]	training's rmse: 0.0307403	valid_1's rmse: 0.610806
[4825]	training's rmse: 0.0307249	valid_1's rmse: 0.610803
[4826]	training's rmse: 0.0307173	valid_1's rmse: 0.610801
[4827]	training's rmse: 0.0307095	valid_1's rmse: 0.6108
[4828]	training's rmse: 0.0307011	valid_1's rmse: 0.610799
[4829]	training's rmse: 0.0306928	valid_1's rmse: 0.610798
[4830]	training's rmse: 0.0306841	valid_1's rmse: 0.610797
[4831]	training's rmse: 0.030681	valid_1's rmse: 0.610798
[4832]	training's rmse: 0.0306753	valid_1's rmse: 0.610798
[4833]	training's rmse: 0.0306716	valid_1's rmse: 0.610798
[4834]	training's rmse: 0.0306662	valid_1's rmse: 0.610797
[4

[4959]	training's rmse: 0.0296689	valid_1's rmse: 0.610739
[4960]	training's rmse: 0.0296626	valid_1's rmse: 0.610739
[4961]	training's rmse: 0.0296586	valid_1's rmse: 0.61074
[4962]	training's rmse: 0.0296465	valid_1's rmse: 0.610739
[4963]	training's rmse: 0.0296312	valid_1's rmse: 0.610743
[4964]	training's rmse: 0.0296279	valid_1's rmse: 0.610743
[4965]	training's rmse: 0.0296213	valid_1's rmse: 0.610743
[4966]	training's rmse: 0.0296175	valid_1's rmse: 0.610743
[4967]	training's rmse: 0.0296108	valid_1's rmse: 0.610743
[4968]	training's rmse: 0.0296048	valid_1's rmse: 0.610744
[4969]	training's rmse: 0.0295846	valid_1's rmse: 0.610743
[4970]	training's rmse: 0.0295822	valid_1's rmse: 0.610743
[4971]	training's rmse: 0.0295795	valid_1's rmse: 0.610743
[4972]	training's rmse: 0.0295708	valid_1's rmse: 0.610742
[4973]	training's rmse: 0.0295604	valid_1's rmse: 0.610742
[4974]	training's rmse: 0.0295542	valid_1's rmse: 0.610741
[4975]	training's rmse: 0.0295506	valid_1's rmse: 0.61074

In [103]:
y_pred_submit = rmse_val.predict(x_test_reduced, num_iteration=rmse_val.best_iteration)

In [169]:
test_id = pd.read_csv('test_grouped_id.csv')

/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [171]:
test_id = test_id.loc[:, ~test_id.columns.str.contains('^Unnamed')]

In [173]:
submission = pd.DataFrame({'fullVisitorId':test_id['fullVisitorId'],'PredictedLogTotalRevenue_perCustomer':y_pred_submit})
submission['fullVisitorId']= submission['fullVisitorId'].astype(str)
submission['PredictedLogTotalRevenue_perCustomer']=submission['PredictedLogTotalRevenue_perCustomer'].apply(lambda x: 0 if x<0 else x)

In [175]:
submission.to_csv('submission_LGBM.csv')

In [176]:
rmse_v12

0.6106969969400216

In [177]:
rmse_t12

0.028971592404011762

In [91]:
rmse_v11

0.6106975016256623

In [92]:
rmse_t11

0.02897159240402142

In [88]:
rmse_v10

0.6110207603960199

In [89]:
rmse_t10

0.023738188273136025

In [85]:
rmse_t9

0.020294617196461973

In [86]:
rmse_v9

0.6273708900068489

In [73]:
rmse_v8

0.6607254141969766

In [74]:
rmse_t8

0.05903756234033532

In [68]:
rmse_v7

0.6581985771200763

In [69]:
rmse_t7

0.25507579206222797

In [61]:
rmse_t6

0.5168227779735912

In [62]:
rmse_v6

0.7167359094023436

In [55]:
rmse_v5

0.8671640628381363

In [56]:
rmse_t5

0.77272212017713

In [50]:
rmse_t4

0.7585244443135454

In [51]:
rmse_v4

0.8553213135732253

In [46]:
rmse_t3

0.5815554234060244

In [47]:
rmse_v3

0.7532258686645664

In [42]:
rmse_t2

0.5987741886405407

In [43]:
rmse_v2

0.7531989950098282

In [35]:
rmse_t1

0.9075212398674616

In [39]:
rmse_v1

0.9414817504736281